<a href="https://colab.research.google.com/github/kirtit2/IS445-Analyzing_Film_Permits/blob/main/IS445_FinalProject_Part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Installing and Importing Libraries:**

In [1]:
#@title
#!pip install sodapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 586 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
#@title
#!pip install geopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#@title
import numpy as np
import pandas as pd

from sodapy import Socrata

import datetime
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import geopy
from geopy.geocoders import Nominatim

import json

## **Loading the Dataset:**

New York Film permits dataset has been taken from: https://data.cityofnewyork.us/City-Government/Film-Permits/tg4x-b46p

API documentation: https://dev.socrata.com/foundry/data.cityofnewyork.us/tg4x-b46p

In [4]:
#Code taken from API Documentation
#!/usr/bin/env python

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("tg4x-b46p", limit=40000)

# Convert to pandas DataFrame
film_permits = pd.DataFrame.from_records(results)

In [5]:
film_permits.head()

,eventid,eventtype,startdatetime,enddatetime,enteredon,eventagency,parkingheld,borough,communityboard_s,policeprecinct_s,category,subcategoryname,country,zipcode_s
0,672776,Shooting Permit,2022-09-15T12:00:00.000,2022-09-15T23:30:00.000,2022-09-13T15:31:30.000,"Mayor's Office of Film, Theatre & Broadcasting",NORMAN AVENUE between MONITOR STREET and NORTH...,Brooklyn,"1, 2","108, 94",Television,Episodic series,United States of America,"11101, 11222"
1,672774,Shooting Permit,2022-09-15T06:00:00.000,2022-09-15T23:59:00.000,2022-09-13T15:28:37.000,"Mayor's Office of Film, Theatre & Broadcasting",NORTH 5 STREET between KENT AVENUE and DEAD...,Brooklyn,"1, 3","81, 94",Commercial,Commercial,United States of America,"11233, 11249"
2,672761,Shooting Permit,2022-09-15T04:00:00.000,2022-09-15T13:00:00.000,2022-09-13T14:33:07.000,"Mayor's Office of Film, Theatre & Broadcasting",WEST 48 STREET between 6 AVENUE and 7 AVENUE,Manhattan,5,18,Television,News,United States of America,"10036, 10105"
3,672733,Shooting Permit,2022-09-15T07:00:00.000,2022-09-15T21:00:00.000,2022-09-13T13:27:59.000,"Mayor's Office of Film, Theatre & Broadcasting",JEWEL STREET between MESEROLE AVENUE and NORMA...,Brooklyn,1,94,Television,Cable-episodic,United States of America,11222
4,672716,Shooting Permit,2022-09-15T06:00:00.000,2022-09-15T23:59:00.000,2022-09-13T12:50:05.000,"Mayor's Office of Film, Theatre & Broadcasting",MONITOR STREET between GREENPOINT AVENUE and N...,Queens,"1, 2","108, 94",Television,Episodic series,United States of America,"11101, 11222"


## **Data Pre-processing and Exploration:**

In [6]:
film_permits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11408 entries, 0 to 11407
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   eventid           11408 non-null  object
 1   eventtype         11408 non-null  object
 2   startdatetime     11408 non-null  object
 3   enddatetime       11408 non-null  object
 4   enteredon         11408 non-null  object
 5   eventagency       11408 non-null  object
 6   parkingheld       11408 non-null  object
 7   borough           11408 non-null  object
 8   communityboard_s  11408 non-null  object
 9   policeprecinct_s  11408 non-null  object
 10  category          11408 non-null  object
 11  subcategoryname   11408 non-null  object
 12  country           11408 non-null  object
 13  zipcode_s         11408 non-null  object
dtypes: object(14)
memory usage: 1.2+ MB


In [7]:
film_permits.isnull().sum()

eventid             0
eventtype           0
startdatetime       0
enddatetime         0
enteredon           0
eventagency         0
parkingheld         0
borough             0
communityboard_s    0
policeprecinct_s    0
category            0
subcategoryname     0
country             0
zipcode_s           0
dtype: int64

In [8]:
film_permits.describe()

,eventid,eventtype,startdatetime,enddatetime,enteredon,eventagency,parkingheld,borough,communityboard_s,policeprecinct_s,category,subcategoryname,country,zipcode_s
count,11408,11408,11408,11408,11408,11408,11408,11408,11408,11408,11408,11408,11408,11408
unique,11408,4,4445,5399,11246,1,6810,5,343,787,9,27,3,1397
top,672776,Shooting Permit,2021-11-18T07:00:00.000,2021-10-25T21:00:00.000,2021-10-08T13:24:11.000,"Mayor's Office of Film, Theatre & Broadcasting",EAST 11 STREET between 3 AVENUE and 4 AVENUE...,Manhattan,1,94,Television,Episodic series,United States of America,11222
freq,1,10162,16,12,5,11408,184,4678,3048,1749,7275,5074,11390,1618


In [9]:
film_permits['count_flag'] = pd.Series([1 for x in range(len(film_permits.index))])

**Converting the datatype of categorical variables from object to category:**

In [10]:
film_permits['eventtype'] = film_permits['eventtype'].astype('category')
film_permits['eventagency'] = film_permits['eventagency'].astype('category')
film_permits['borough'] = film_permits['borough'].astype('category')
film_permits['category'] = film_permits['category'].astype('category')
film_permits['subcategoryname'] = film_permits['subcategoryname'].astype('category')
film_permits['country'] = film_permits['country'].astype('category')


**Converting the datatype of date variables from object to datetime:**

In [11]:
film_permits['enddatetime']=pd.to_datetime(
                                  film_permits['enddatetime'],
                                 format='%Y/%m/%d %H:%M:%S')

film_permits['startdatetime']=pd.to_datetime(
                                  film_permits['startdatetime'],
                                 format='%Y/%m/%d %H:%M:%S')

In [12]:
film_permits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11408 entries, 0 to 11407
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   eventid           11408 non-null  object        
 1   eventtype         11408 non-null  category      
 2   startdatetime     11408 non-null  datetime64[ns]
 3   enddatetime       11408 non-null  datetime64[ns]
 4   enteredon         11408 non-null  object        
 5   eventagency       11408 non-null  category      
 6   parkingheld       11408 non-null  object        
 7   borough           11408 non-null  category      
 8   communityboard_s  11408 non-null  object        
 9   policeprecinct_s  11408 non-null  object        
 10  category          11408 non-null  category      
 11  subcategoryname   11408 non-null  category      
 12  country           11408 non-null  category      
 13  zipcode_s         11408 non-null  object        
 14  count_flag        1140

**Creating new variables which are:**
1. start_year: start year of film permit
2. end_year: end year of film permit
3. start_month: start month of film permit
4. end_month: end month of film permit
5. start_weekday: start weekday of film permit
6. end_weekday: end weekday of film permit
7. difference_day: total number of days for the permit

In [13]:
film_permits['end_year'] = film_permits['enddatetime'].dt.year
film_permits['start_year'] = film_permits['startdatetime'].dt.year

film_permits['end_month'] = film_permits['enddatetime'].dt.month
film_permits['start_month'] = film_permits['startdatetime'].dt.month

film_permits['start_weekday'] = film_permits['startdatetime'].dt.weekday
film_permits['end_weekday'] = film_permits['enddatetime'].dt.weekday

In [14]:
film_permits['start_year'].unique()

array([2022, 2021])

In [15]:
film_permits['difference_day'] = pd.DataFrame((film_permits['enddatetime']-film_permits['startdatetime']).values/np.timedelta64(1,'D'), columns=['difference_day']).round(2)

In [16]:
film_permits.head()

,eventid,eventtype,startdatetime,enddatetime,enteredon,eventagency,parkingheld,borough,communityboard_s,policeprecinct_s,...,country,zipcode_s,count_flag,end_year,start_year,end_month,start_month,start_weekday,end_weekday,difference_day
0,672776,Shooting Permit,2022-09-15 12:00:00,2022-09-15 23:30:00,2022-09-13T15:31:30.000,"Mayor's Office of Film, Theatre & Broadcasting",NORMAN AVENUE between MONITOR STREET and NORTH...,Brooklyn,"1, 2","108, 94",...,United States of America,"11101, 11222",1,2022,2022,9,9,3,3,0.48
1,672774,Shooting Permit,2022-09-15 06:00:00,2022-09-15 23:59:00,2022-09-13T15:28:37.000,"Mayor's Office of Film, Theatre & Broadcasting",NORTH 5 STREET between KENT AVENUE and DEAD...,Brooklyn,"1, 3","81, 94",...,United States of America,"11233, 11249",1,2022,2022,9,9,3,3,0.75
2,672761,Shooting Permit,2022-09-15 04:00:00,2022-09-15 13:00:00,2022-09-13T14:33:07.000,"Mayor's Office of Film, Theatre & Broadcasting",WEST 48 STREET between 6 AVENUE and 7 AVENUE,Manhattan,5,18,...,United States of America,"10036, 10105",1,2022,2022,9,9,3,3,0.38
3,672733,Shooting Permit,2022-09-15 07:00:00,2022-09-15 21:00:00,2022-09-13T13:27:59.000,"Mayor's Office of Film, Theatre & Broadcasting",JEWEL STREET between MESEROLE AVENUE and NORMA...,Brooklyn,1,94,...,United States of America,11222,1,2022,2022,9,9,3,3,0.58
4,672716,Shooting Permit,2022-09-15 06:00:00,2022-09-15 23:59:00,2022-09-13T12:50:05.000,"Mayor's Office of Film, Theatre & Broadcasting",MONITOR STREET between GREENPOINT AVENUE and N...,Queens,"1, 2","108, 94",...,United States of America,"11101, 11222",1,2022,2022,9,9,3,3,0.75


## **Data Visualizations:**

In [17]:
from plotly.offline import plot
from plotly.offline import iplot
import plotly.graph_objs as go

In [18]:
trace = go.Histogram(y=film_permits['category'])
data = [trace]
plot(data)

'temp-plot.html'

### **Number of permits granted in each category of shooting activity:**

In [19]:
film_permits['category'].value_counts()

Television           7275
Film                 1152
Theater              1095
Commercial            820
Still Photography     507
WEB                   424
Documentary            68
Student                37
Music Video            30
Name: category, dtype: int64

In [20]:
fig = px.histogram(film_permits, x='category', color = 'borough').update_xaxes(categoryorder = "total descending")
fig.update_layout(title="Number of permits granted in each category of shooting activity")
fig.show()

The top 3 categories are television, film, theatre. Television alone has more than half of the total permits. Double clicking on any of the boroughs filters the graph to show data for the selected borough.

### **Number of permits granted in each borough**

In [21]:
film_permits['borough'].value_counts()

Manhattan        4678
Brooklyn         4085
Queens           2080
Bronx             465
Staten Island     100
Name: borough, dtype: int64

In [22]:
fig = px.histogram(film_permits, x='borough', color = 'category').update_xaxes(categoryorder = "total descending")
fig.update_layout(title="Number of permits granted in each borough for a shooting activity")
fig.show()

### **Number of permits granted per event type:**

In [23]:
film_permits['eventtype'].value_counts()

Shooting Permit                  10162
Theater Load in and Load Outs     1062
Rigging Permit                     119
DCAS Prep/Shoot/Wrap Permit         65
Name: eventtype, dtype: int64

In [24]:
fig = px.histogram(film_permits, x='eventtype').update_xaxes(categoryorder = "total descending")
fig.update_layout(title="Number of permits granted per event type")
fig.show()

The highest permits are for shooting. In fact, when compared with other event types shooting has a stark majority clearly. We can hover over the chart to see the exact number of permits for each event type.

### **Total days in Permit per event type:**

In [25]:
fig = px.pie(film_permits, values='difference_day', names='eventtype', title='Distribution of total days in permit per event type')
fig.show()

Given that shooting permits are the highest, it is expected that the total days in "shooting_permit" are also higher. Even though the Theatre Load in and Load out permits are less in number, still they make up over 27% of total days in permit. This implies that, the "Theatre Load in and Load Outs" permit span for lnger time duration.

### **Total permits by country:**

In [26]:
film_permits['country'].value_counts()

United States of America    11390
United Kingdom                 12
Canada                          6
Name: country, dtype: int64

In [27]:
film_permits_country = film_permits.groupby(['start_month','country'])['count_flag'].sum().reset_index()
film_permits_country.rename(columns = {'count_flag': 'total_permits'}, inplace = True)
film_permits_country.head()

,start_month,country,total_permits
0,1,Canada,0
1,1,United Kingdom,0
2,1,United States of America,786
3,2,Canada,0
4,2,United Kingdom,0


In [28]:
fig = px.line(film_permits_country, x="start_month",y='total_permits', color='country')
fig.show()

Using a line chart is not the best idea to visualize this as the data for UK and Canada is overlapping and is very less in number hence is seen to be 0 from the above visualization.
We will try using a stacked bar chart for better inference.

In [29]:
fig = px.histogram(film_permits_country, x="start_month", y="total_permits",
             color='country', barmode='group',
             histfunc='max',
             title='Total Permits by country',)
fig.show()

Yet again, because the values for UK and Canada are very low compared to US they are hardly to be seen on the graph.

### **Top 5 Categories by Month**

In [30]:
film_permits['category'].value_counts()

Television           7275
Film                 1152
Theater              1095
Commercial            820
Still Photography     507
WEB                   424
Documentary            68
Student                37
Music Video            30
Name: category, dtype: int64

In [31]:
top_category_df = film_permits[film_permits['category'].isin({'Television', 'Film', 'Theater', 'Commercial', 'Still Photography'})]

top_category_df['category'].value_counts()

Television           7275
Film                 1152
Theater              1095
Commercial            820
Still Photography     507
Documentary             0
Music Video             0
Student                 0
WEB                     0
Name: category, dtype: int64

In [32]:
top_category = top_category_df.groupby(['start_month','category',])['count_flag'].sum().reset_index()

top_category = top_category[top_category['category'].isin({'Television', 'Film', 'Theater', 'Commercial', 'Still Photography'})]
top_category.head()

,start_month,category,count_flag
0,1,Commercial,54
2,1,Film,33
4,1,Still Photography,10
6,1,Television,616
7,1,Theater,44


In [33]:
fig = px.line(top_category, x="start_month",y='count_flag', color='category')
fig.show()

The purpose of this visualization was to check for any popular months or seasons across the year when most permits are issued. We do see a spike in the months of March and August. This is especially true for Televesion permits.
Also, we do observe a spike in the film permits issued in May and August.

### **Approval Duration**

In [34]:
approval_df = film_permits.copy()

In [35]:
from ipywidgets import widgets, interactive

In [36]:
approval_df['enteredon']=pd.to_datetime(
                                  approval_df['enteredon'],
                                 format='%Y/%m/%d %H:%M:%S')

approval_df.head()

,eventid,eventtype,startdatetime,enddatetime,enteredon,eventagency,parkingheld,borough,communityboard_s,policeprecinct_s,...,country,zipcode_s,count_flag,end_year,start_year,end_month,start_month,start_weekday,end_weekday,difference_day
0,672776,Shooting Permit,2022-09-15 12:00:00,2022-09-15 23:30:00,2022-09-13 15:31:30,"Mayor's Office of Film, Theatre & Broadcasting",NORMAN AVENUE between MONITOR STREET and NORTH...,Brooklyn,"1, 2","108, 94",...,United States of America,"11101, 11222",1,2022,2022,9,9,3,3,0.48
1,672774,Shooting Permit,2022-09-15 06:00:00,2022-09-15 23:59:00,2022-09-13 15:28:37,"Mayor's Office of Film, Theatre & Broadcasting",NORTH 5 STREET between KENT AVENUE and DEAD...,Brooklyn,"1, 3","81, 94",...,United States of America,"11233, 11249",1,2022,2022,9,9,3,3,0.75
2,672761,Shooting Permit,2022-09-15 04:00:00,2022-09-15 13:00:00,2022-09-13 14:33:07,"Mayor's Office of Film, Theatre & Broadcasting",WEST 48 STREET between 6 AVENUE and 7 AVENUE,Manhattan,5,18,...,United States of America,"10036, 10105",1,2022,2022,9,9,3,3,0.38
3,672733,Shooting Permit,2022-09-15 07:00:00,2022-09-15 21:00:00,2022-09-13 13:27:59,"Mayor's Office of Film, Theatre & Broadcasting",JEWEL STREET between MESEROLE AVENUE and NORMA...,Brooklyn,1,94,...,United States of America,11222,1,2022,2022,9,9,3,3,0.58
4,672716,Shooting Permit,2022-09-15 06:00:00,2022-09-15 23:59:00,2022-09-13 12:50:05,"Mayor's Office of Film, Theatre & Broadcasting",MONITOR STREET between GREENPOINT AVENUE and N...,Queens,"1, 2","108, 94",...,United States of America,"11101, 11222",1,2022,2022,9,9,3,3,0.75


In [37]:
# Referred from https://www.easytweaks.com/pandas-difference-between-dates/
approval_df['approval_duration'] = pd.DataFrame((approval_df['startdatetime']-approval_df['enteredon']).values/np.timedelta64(1,'D'), columns=['difference_day']).round(2)
approval_df.head()

,eventid,eventtype,startdatetime,enddatetime,enteredon,eventagency,parkingheld,borough,communityboard_s,policeprecinct_s,...,zipcode_s,count_flag,end_year,start_year,end_month,start_month,start_weekday,end_weekday,difference_day,approval_duration
0,672776,Shooting Permit,2022-09-15 12:00:00,2022-09-15 23:30:00,2022-09-13 15:31:30,"Mayor's Office of Film, Theatre & Broadcasting",NORMAN AVENUE between MONITOR STREET and NORTH...,Brooklyn,"1, 2","108, 94",...,"11101, 11222",1,2022,2022,9,9,3,3,0.48,1.85
1,672774,Shooting Permit,2022-09-15 06:00:00,2022-09-15 23:59:00,2022-09-13 15:28:37,"Mayor's Office of Film, Theatre & Broadcasting",NORTH 5 STREET between KENT AVENUE and DEAD...,Brooklyn,"1, 3","81, 94",...,"11233, 11249",1,2022,2022,9,9,3,3,0.75,1.61
2,672761,Shooting Permit,2022-09-15 04:00:00,2022-09-15 13:00:00,2022-09-13 14:33:07,"Mayor's Office of Film, Theatre & Broadcasting",WEST 48 STREET between 6 AVENUE and 7 AVENUE,Manhattan,5,18,...,"10036, 10105",1,2022,2022,9,9,3,3,0.38,1.56
3,672733,Shooting Permit,2022-09-15 07:00:00,2022-09-15 21:00:00,2022-09-13 13:27:59,"Mayor's Office of Film, Theatre & Broadcasting",JEWEL STREET between MESEROLE AVENUE and NORMA...,Brooklyn,1,94,...,11222,1,2022,2022,9,9,3,3,0.58,1.73
4,672716,Shooting Permit,2022-09-15 06:00:00,2022-09-15 23:59:00,2022-09-13 12:50:05,"Mayor's Office of Film, Theatre & Broadcasting",MONITOR STREET between GREENPOINT AVENUE and N...,Queens,"1, 2","108, 94",...,"11101, 11222",1,2022,2022,9,9,3,3,0.75,1.72


In [38]:
x_dropdown = widgets.Dropdown (options =['borough', 'eventtype', 'category'], description='X-Axis:')

grouped_dropdown = widgets.Dropdown (options =['eventtype', 'category', 'borough'], description='Color group:')

year_dropdown = widgets.Dropdown (options =['All'] + list(approval_df['start_year'].unique()), description='Year:', value='All')


def plot_approval(x_selected, grouped_selected, year_selected):
    temp_df = approval_df.copy()
    if year_selected != 'All':
        temp_df = temp_df[temp_df['start_year'] == year_selected]
    
    # Referred from https://plotly.com/python/bar-charts/
    fig = px.histogram(temp_df, x=x_selected, y="approval_duration",
                 color=grouped_selected, barmode='group',
                 histfunc='max',
                 height=400)
    fig.update_layout(
        title='Approval duration Analysis',
        xaxis=dict(
            title='Boroughs'
        ),
        yaxis=dict(
            title='Maximum Approval duration'
        )
    )
    fig.show()

interactive(plot_approval, x_selected=x_dropdown, grouped_selected = grouped_dropdown, year_selected=year_dropdown)

interactive(children=(Dropdown(description='X-Axis:', options=('borough', 'eventtype', 'category'), value='bor…

Theater filming permit in Manhattan has the highest lead time. Staten Island and Queens either doesn't have a lot of theatre related filming or the permit duration is lesser. Bronx has higher lead time for shooting permits compared to other boroughs.

In [39]:
# Referred from https://plotly.com/python/bubble-charts/
bubble_fig = px.scatter(approval_df, x="difference_day", y="borough" , color="eventtype",
                 hover_name="borough", log_x=True, size="approval_duration")
bubble_fig.update_layout(
        title='Approval lead time v/s Duration of the event',
        xaxis=dict(
            title='Duration of the event'
        ),
        yaxis=dict(
            title='Boroughs'
        )
    )
bubble_fig.show()

As observed, there is no relation between the length of the filming to the approval time for the permit. We can see the lower approval time for the event with larger duration.

### **Lat/long geomaps:**

To extract the Latitude and Longitude coordinates of the places, we will be utilizing the Street Address given in the data.

In [50]:
film_permits['parkingheld'].value_counts().to_frame().head(10)

,parkingheld
"EAST 11 STREET between 3 AVENUE and 4 AVENUE, 3 AVENUE between EAST 11 STREET and EAST 12 STREET",184
"FROST STREET between DEBEVOISE AVENUE and MORGAN AVENUE, DEBEVOISE AVENUE between FROST STREET and RICHARDSON STREET",156
WEST 48 STREET between 6 AVENUE and 7 AVENUE,153
"NORTH HENRY STREET between GREENPOINT AVENUE and MESEROLE AVENUE, RUSSELL STREET between GREENPOINT AVENUE and CALYER STREET",139
"WEST 55 STREET between 11 AVENUE and 12 AVENUE, WEST 56 STREET between 11 AVENUE and 12 AVENUE, 11 AVENUE between WEST 55 STREET and WEST 56 STREET",119
"QUEENS PLAZA SOUTH between 21 STREET and 22 STREET, 22 STREET between QUEENS PLAZA SOUTH and 43 AVENUE",94
"GREENPOINT AVENUE between WEST STREET and DEAD END, KENT STREET between WEST STREET and DEAD END, JAVA STREET between WEST STREET and DEAD END",88
"EAGLE STREET between FRANKLIN STREET and WEST STREET, WEST STREET between EAGLE STREET and FREEMAN STREET, FREEMAN STREET between WEST STREET and FRANKLIN STREET",85
"COVERT ST between WYCKOFF AVENUE and IRVING AVENUE, IRVING AVE between ELDERT ST and COVERT ST",74
"WEST 31 STREET between 7 AVENUE and 8 AVENUE, 8 AVENUE between WEST 31 STREET and WEST 33 STREET",68


In [42]:
geolocator = Nominatim()
street_address_list = []
lat_long_list = []

/usr/local/lib/python3.8/dist-packages/geopy/geocoders/osm.py:132: UserWarning:

Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.



Only grabbing the Latitude and Longitude of the Top 15 locations.

In [51]:
parking_details = film_permits['parkingheld'].value_counts().head(15)
parking_df = pd.DataFrame(list(parking_details.items()), columns=['parkingheld','permit_count'])
#parking_df

In [47]:
# !pip install humanize

In [52]:
import re
import humanize
for street_info in parking_df['parkingheld']:
    street_address = street_info.split('between')[0].strip()
    found_numbers = re.search(r'\d+', street_address)
    if found_numbers is not None:
        indices = list(found_numbers.span())
        street_number = street_address[indices[0]:indices[1]]
        street_parts = street_address.split(street_number)
        street_address = street_parts[0] + humanize.ordinal(street_number) + street_parts[1] + ', New York City, New York'
    else:
        street_address = street_address + ', New York City, New York'
    location_dict = geolocator.geocode(street_address).raw
    latitude = float(location_dict['lat'])
    longitude = float(location_dict['lon'])
    street_address_list.append(street_address)
    lat_long_list.append([latitude,longitude])

new_df = pd.DataFrame({'parkingheld':street_address_list})
parking_df.update(new_df)
parking_df['lat_long'] = lat_long_list


In [53]:
# parking_df

In [54]:
df = parking_df.lat_long.apply(pd.Series)
df.columns = ['latitude', 'longitude']
df1 = parking_df["parkingheld"]
df = df.join(df1)
df2 = parking_df["permit_count"]
df = df.join(df2)
df

,latitude,longitude,parkingheld,permit_count
0,40.587506,-73.959354,"EAST 11th STREET, New York City, New York",184
1,40.717894,-73.946796,"FROST STREET, New York City, New York",156
2,40.763376,-73.992579,"WEST 48th STREET, New York City, New York",153
3,40.729442,-73.944186,"NORTH HENRY STREET, New York City, New York",139
4,40.765354,-73.983599,"WEST 55th STREET, New York City, New York",119
5,40.754243,-73.949216,"QUEENS PLAZA SOUTH, New York City, New York",94
6,40.733160,-73.940678,"GREENPOINT AVENUE, New York City, New York",88
7,40.734682,-73.960130,"EAGLE STREET, New York City, New York",85
8,40.689253,-73.909413,"COVERT ST, New York City, New York",74
9,40.575145,-73.997482,"WEST 31st STREET, New York City, New York",68


In [55]:
# df['permit_count'].sum()

In [56]:
# https://python-visualization.github.io/folium/modules.html
#https://www.analyticsvidhya.com/blog/2020/06/guide-geospatial-analysis-folium-python/
import folium

In [57]:
folium_map = folium.Map(location=parking_df.iloc[0]['lat_long'],
                        zoom_start=12)
for curr_loc in list(parking_df.index):
    folium.Marker(location=parking_df.iloc[curr_loc]['lat_long'],
                  popup=f"ParkingHeld:<br>{parking_df.iloc[curr_loc]['parkingheld']}<br>"
                        f"Permit_Count:<br>{parking_df.iloc[curr_loc]['permit_count']}<br>"
                 ).add_to(folium_map)
    folium_map.add_child(folium.ClickForMarker(popup='Camp Muir'))

folium_map.add_child(folium.TileLayer('Stamen Terrain'))
folium_map.add_child(folium.TileLayer('Stamen Toner'))
folium_map.add_child(folium.TileLayer('Stamen Water Color'))
folium_map.add_child(folium.TileLayer('cartodbpositron'))
folium_map.add_child(folium.TileLayer('cartodbdark_matter'))
folium_map.add_child(folium.LayerControl())

folium_map


* The top 15 filming locations with the different views (Filter on Top Right corner) can be seen in the map above.

* East 11th STREET, New York City, New York near Union Square is the popular place.

### **Timing Permits:**




In [59]:
fig = px.histogram(film_permits, x="start_month",height=400)
fig.update_layout(
    title='Start Date Occurrence Frequency',
    xaxis=dict(
        title='Start Month'
    ),
    yaxis=dict(
        title='Frequency'
    )
)
fig.show()


* It seems that the most popular month for the film permits to begin is the end of First quarter and middle of Third quarter. 

* It can also be deduced that March to August have fairly decent distribution since those are the good weather days for the shoots to take place.

# Analysis Results



*   The main finding from these data is that we can determine when, where and kind of different shooting activities take place in New York City.
*   Television receives the most permit requests, which makes sense given that it shoots for a longer time and makes extensive use of the environment.
*  We also found that March and August were the most popular months for the permits. 
* Lastly, there was no relation found between the approval time for the different shooting activities requiring permits.

